<img width="8%" alt="LangChain.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LangChain.png" style="border-radius: 15%">

# LangChain - Create Agent
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LangChain+-+Create+Agent:+Error+short+description">Bug report</a>

**Tags:** #langchain #neo4j #graph #knowledge

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2024-03-25 (Created: 2024-03-25)

**Description:** This notebook creates an Agent.

**References:**
- [Medium - Using a Knowledge Graph to Implement a DevOps RAG Application](https://medium.com/neo4j/using-a-knowledge-graph-to-implement-a-devops-rag-application-b6ba24831b16)

## Input

### Import libraries

In [1]:
import naas
import os
import requests
try:
    from langchain.graphs import Neo4jGraph
    from langchain.vectorstores.neo4j_vector import Neo4jVector
    from langchain.chains import RetrievalQA, GraphCypherQAChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
except:
    !pip install langchain==0.1.13 --user
    from langchain.graphs import Neo4jGraph
    from langchain.vectorstores.neo4j_vector import Neo4jVector
    from langchain.chains import RetrievalQA, GraphCypherQAChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
try:
    from langchain_openai import OpenAIEmbeddings
    from langchain_openai import ChatOpenAI
except:
    !pip install langchain-openai=0.1.1 --user
    from langchain_openai import OpenAIEmbeddings
    from langchain_openai import ChatOpenAI
try:
    import neo4j
except:
    !pip install neo4j==5.18.0 --user
    import neo4j
from neo4j import GraphDatabase
import pandas as pd

### Setup variables

In [4]:
url = "neo4j+s://a23aa9f6.databases.neo4j.io:7687"
username = naas.secret.get("NEO4J_USERNAME")
password = naas.secret.get("NEO4J_PASSWORD")
graph_source_url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/e90b0c9386bf8be15b199e8ac8f83fc265a2ac57/microservices.json"
os.environ['OPENAI_API_KEY'] = naas.secret.get("OPENAI_API_KEY")

## Model

### Neo4j Graph Setup

In [5]:
graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)
graph

### Get graph dataset

In [6]:
import_query = requests.get(graph_source_url).json().get('query')
import_query

### Import graph into Neo4j

In [7]:
graph.query(
    import_query
)

## Output

### Vector Index & Retrieval QA

In [8]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)

In [9]:
vector_qa.invoke(
    "How will recommendation service be updated?"
)

In [10]:
vector_qa.invoke(
    "How many open tickets there are?"
)

### GraphCypherQAChain

In [11]:
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [12]:
cypher_chain.invoke(
    "How will recommendation service be updated?"
)

In [13]:
cypher_chain.invoke(
    "How many open tickets there are?"
)

### Knowledge Graph Agent

In [14]:
tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions.
        Not useful for counting.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of 
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools, 
    ChatOpenAI(temperature=0, model_name='gpt-4'),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

In [15]:
response = mrkl.invoke("How will recommendation service be updated?")
print(response)

In [ ]:
response = mrkl.invoke("How many open tickets there are?")
print(response)

In [ ]:
response = mrkl.invoke("What is the latest post about data product?")
print(response)

In [ ]:
response = mrkl.invoke("What is the name of the teams?")
print(response)

In [ ]:
response = mrkl.invoke("What is Alice's team?")
print(response)

In [ ]:
response = mrkl.invoke("What is Frank's team?")
print(response)